In [1]:
%matplotlib inline

In [2]:
import sys
sys.path.append('/Users/yuxuan/Documents/Research_module')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
from despotic import cloud

In [4]:
# Start with a Milky Way GMC; set dust temperature to 10 K
#gmc = cloud(os.path.join(os.environ['DESPOTIC_HOME'],
#                         'cloudfiles/MilkyWayGMC.desp'))
gmc = cloud(os.path.join('/Users/yuxuan/Documents/Research_module/despotic_full/despotic/cloudfiles/MilkyWayGMC.desp'))
gmc.Td = 10.0
gmc.rad.TradDust = 10.0

In [5]:
# Parameters for variable temperature case
gmc.nH = 1000.
gmc.NH = 1.0e23
gmc.rad.chi = 10.0
gmc.rad.ionRate = 1.0e-16

In [6]:
# Define grids in density and velocity gradient
lognH = np.linspace(0,10,100)
logdVdr = np.linspace(-3,3,75)

In [7]:
# generate optically thin(only with RT, not Level population)
sp1 = 'CO'
sp2 = 'co'
# CO abundances and gas temperatures to use
x = [1.0e-4]
Tgval = [ 10.]
labels = ['std']

In [8]:
# Generate tables for constant temperature cases
for i, x_, Tgval_, lab in zip(range(len(x)), x, Tgval, labels):
    
    # Print status
    print("Generating co_thin2_lum_{:s}.txt for x = {:e}, Tg = {:f}"
          .format(lab, x_, Tgval_))
    
    # Set abundance and gas temperature
    gmc.addEmitter(sp1, x_)
    gmc.Tg = Tgval_
    
    # Compute luminosity on grid
    loglum = []
    for lognH_ in lognH:
        gmc.nH = 10.**lognH_
        loglum1 = []
        for logdVdr_ in logdVdr:
            gmc.dVdr = 10.**logdVdr_ * 1.0e5 / 3.09e18
            l = gmc.lineLum(sp1, escapeProbGeom='LVG', lumOnly=True, 
                            transition=[np.array([1]), np.array(0)])[0]
            l = gmc.lineLum(sp1, escapeProbGeom='LVG', lumOnly=True, 
                            transition=[np.array([1]), np.array(0)], thin=True, noRecompute=True)[0]
            loglum1.append(np.log10(l))
        loglum.append(loglum1)
    loglum = np.array(loglum)
    
    # Save outputs
    if i == 0:
        np.savetxt('%s_n.txt'%(sp2), lognH)
        np.savetxt('%s_dvdr.txt'%(sp2), logdVdr)
    np.savetxt('CO_thin2/%s_thin2_lum_%s.txt'%(sp2,lab), loglum)

Generating co_thin2_lum_std.txt for x = 1.000000e-04, Tg = 10.000000


/Users/yuxuan/Documents/Research_module/despotic/emitter.py:840: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  self.levPop, res, rank, s = np.linalg.lstsq(m, b)
